In [3]:
#Load gsheets libraries
import gspread 
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#authenticate gsheets
google_key_file = 'service_key.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [4]:
spreadsheet_key = '1auoQ9XanosnM7RUInzqeZi9EIgwtCtmtubNpXrfF6OM' 
wks_name = 'sentimentAnalysis'


book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet(wks_name) 

In [5]:
# Load twitter credentials
with open("covid19-sentanalysis-twitter_credentials.json") as datafile:
  data = json.load(datafile)

NameError: name 'json' is not defined

In [ ]:
import tweepy
from textblob import TextBlob


# Step 1 - Authenticate twitter
consumer_key= data['consumer_key'] #'API_CONSUMER_KEY_HERE'
consumer_secret=  data['consumer_secret']#'CONSUMER_SECRET_HERE'

access_token= data['access_token_key'] #'ACCESS_TOKEN_HERE'
access_token_secret= data['access_token_secret'] #'ACCESS_TOKEN_SECRET_HERE'



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# create API, set limits to avouid errors because of a timeout 
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

#Print 5 tweets for testing purposes - Should be deleted afterwards
home_tweets = api.home_timeline(count=5)
print("printing tweets from timeline \n ")
for tweet in home_tweets:
    print(tweet.text)
    print("")

**GET Twitter Stream and Do Sentiment Analysis in Real time**


In [ ]:
from tweepy import Stream
from tweepy import StreamListener
#from google.colab import files
import pandas as pd
import json
import re
import csv
import nltk
nltk.download('punkt')

In [ ]:
trump = 0
warren = 0

header_name = ['Trump', 'Warren','Text','id', 'created_at', 'user_id','coordinates','location']
with open('sentiments.csv', 'w') as file:
  writer = csv.DictWriter(file, fieldnames = header_name)
  writer.writeheader()

# sentiments_df = pd.DataFrame(columns=['Trump','Warren'])
#api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

class Listener(StreamListener):
  def __init__(self):
    super().__init__()
    self.max_tweets = 10
    self.tweet_count = 0
    self.tweet_list = []
    
  def on_data(self, data):
    raw_twitts = json.loads(data)
    try:
      #  Fields we need: id, created_at, text, coordinates, author_id
      full_tweets = raw_twitts.copy()
   
      # TO DO: we must drop from full_tweets the tweets that consist only on RT, numbers, etc (see regex used below)
      
            
      tweets = raw_twitts['text']
      tweets = ' '.join(re.sub("(@[A-Za-z0-9]+) | ({*0-9A-Za-z \t]) |] (\wt:\/\/\St+)", " ", tweets).split())
      tweets = ' '.join(re.sub('RT',' ', tweets).split())  
        

      blob = TextBlob(tweets.strip())
      global trump
      global warren

      trump_sentiment = 0
      warren_sentiment = 0

      for sent in blob.sentences:
        if "Trump" in sent and "Warren" not in sent:
          trump_sentiment = trump_sentiment + sent.sentiment.polarity
        else:
          warren_sentiment = warren_sentiment + sent.sentiment.polarity
  
      trump = trump + trump_sentiment
      warren = warren + warren_sentiment
        
        
      info = {'Trump': trump,'Warren': warren, 'Text':raw_twitts['text']
               ,'id':raw_twitts['id'], 'created_at':raw_twitts['created_at']
               , 'user_id':raw_twitts['user']['id'], 'coordinates':raw_twitts['coordinates']
                , 'location':raw_twitts['user']['location']
               }
      self.tweet_list.append(info)
    
      print (tweets,'\n')    
    except:
      print('ERROR got')
    else:
      self.tweet_count+=1
    
      # Once it reaches a fix limit the Write the data into gsheets
      if(self.tweet_count==self.max_tweets):          
        # save to a dataframe for eeasier file upload
        df_tweet_list = pd.DataFrame(self.tweet_list, columns = header_name)
        
        d2g.upload(df_tweet_list, spreadsheet_key, wks_name, credentials=credentials, row_names=False)
        
        print("completed")
        return(False)
      else:
       decoded = json.loads(data)

  def on_error(self, status):
    print(status)

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
twitter_stream = Stream(auth, Listener())
twitter_stream.filter(track = ['Trump','Warren'])